In [ ]:
!pip install Flask scikit-learn
!pip install pyngrok
!ngrok authtoken 2UzFYH3ldSLRwaH04f1POpzrBzf_49rXtPHhDrYevju285QdF

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import pickle
from flask import Flask, request, render_template
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
template_folder = '/gdrive/MyDrive/SunBase/template'

In [ ]:
import socket
socket.gethostbyname(socket.gethostname())

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
app = Flask(__name__,template_folder=template_folder)
run_with_ngrok(app)

@app.route("/")
def home_page():
    return render_template('home.html')

@app.route("/", methods=['POST'])
def predict():

    Name = request.form['Name']
    Age = float(request.form['Age'])
    Gender = request.form['Gender']
    Location = request.form['Location']
    Subscription_Length_Months = float(request.form['Subscription_Length_Months'])
    Monthly_Bill = float(request.form['Monthly_Bill'])
    Total_Usage_GB = float(request.form['Total_Usage_GB'])
    Churn = request.form['Churn']

    model = pickle.load(open('Model.save', 'rb'))
    data = [[Name, Age, Gender, Location, Subscription_Length_Months, Monthly_Bill, Total_Usage_GB, Churn]]
    df = pd.DataFrame(data, columns=['Name', 'Age', 'Gender', 'Location', 'Subscription_Length_Months', 'Monthly_Bill', 'Total_Usage_GB', 'Churn'])

    categorical_feature = {feature for feature in df.columns if df[feature].dtypes == 'O'}

    encoder = LabelEncoder()
    for feature in categorical_feature:
        df[feature] = encoder.fit_transform(df[feature])

    single = model.predict(df)
    probability = model.predict_proba(df)[:, 1]
    probability = probability*100

    if single == 1:
        op1 = "This Customer is likely to Churn!"
        op2 = f"Confidence level is {np.round(probability[0], 2)}"
    else:
        op1 = "This Customer is likely to continue!"
        op2 = f"Confidence level is {np.round(probability[0], 2)}"

    return render_template("home.html", op1=op1, op2=op2,
                           Name=request.form['Name'],
                           Age=request.form['Age'],
                           Gender=request.form['Gender'],
                           Location=request.form['Location'],
                           Subscription_Length_Months=request.form['Subscription_Length_Months'],
                           Monthly_Bill=request.form['Monthly_Bill'],
                           Total_Usage_GB=request.form['Total_Usage_GB'],
                           Churn=request.form['Churn'])


if __name__ == '__main__':
      app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://009d-34-136-213-121.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


ERROR:__main__:Exception on / [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "<ipython-input-24-f1cf7f415eb9>", line 6, in home_page
    return render_template('home.html')
  File "/usr/local/lib/python3.10/dist-packages/flask/templating.py", line 146, in render_template
    template = app.jinja_env.get_or_select_template(template_name_or_list)
  File "/usr/local/lib/python3.10/dist-packages/jinja2/environment.py", line 1081